In [181]:
import pandas as pd
import os
import sys
from IPython.display import display
from transformers import  DistilBertTokenizerFast
import numpy as np
import json

In [182]:
path_text = "../data/train/text/"
path_tsv = "../data/train/tsv/"
all_files = os.listdir(path_text)
tsv_files = os.listdir(path_tsv)

In [183]:
texts = []
text_labels = []
exceptions = []
for file in all_files:
    try:
        with open(os.path.join(path_text,file)) as f:
            text = f.read()
        tsv_data = pd.read_csv(os.path.join(path_tsv,file.split('.')[0]+".tsv"),sep="\t")[['annotType','startOffset','endOffset','text','annotId','other']].sort_values(by='startOffset',ascending=False)
        texts.append(text)
        text_labels.append(tsv_data.values.tolist())
    except Exception as e:
        exceptions.append(e)
        
        
                               

In [184]:
len(texts),len(text_labels)

(233, 233)

In [185]:
entity_labels={
    'Quantity':'QUANTITY',
    'MeasuredEntity': 'MEASURED_ENTITY',
    'MeasuredProperty': 'MEASURED_PROPERTY',
    'Qualifier': 'QUALIFIER'
    
    
}
    

entity_labels

{'Quantity': 'QUANTITY',
 'MeasuredEntity': 'MEASURED_ENTITY',
 'MeasuredProperty': 'MEASURED_PROPERTY',
 'Qualifier': 'QUALIFIER'}

In [186]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

In [191]:
encodings = tokenizer(texts,return_offsets_mapping=True, padding=True, truncation=True)
offset_mappings = encodings.offset_mapping

In [192]:
labels =[]
for offset_mapping,text_label in zip(offset_mappings,text_labels):
    label = BOI_labels(text_label,offset_mapping,entity_labels)
    labels.append(label)
encodings['labels'] = labels

In [177]:
#BOIES
def BOIES_labels(lab_map,offset_mapping,entity_labels):
    labels =[]
    cur = lab_map.pop()
    prev=False
    for token in offset_mapping:
        #If first and second both match, then 'S'
        if cur[1]==token[0] and cur[2]==token[1]:
            labels.append("S-" + entity_labels[cur[0]])
            #pop label
            temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
            while temp[1]==cur[1]:
                cur = temp
                temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
            cur = temp


        #Elseif first not match and prev==False, then 'O'
        elif cur[1]!=token[0] and not prev:
            labels.append("O" )

        #Elseif first match and second dont match,then 'B'
        elif cur[1]==token[0] and cur[2]!=token[1]:
            labels.append("B-" + entity_labels[cur[0]])
            #set prev=True
            prev = True

        #Elseif first doesn't match,prev==True, and second doesn't match then 'I'
        elif cur[1]!=token[0] and cur[2]!=token[1] and prev:
            labels.append("I-" + entity_labels[cur[0]])

        #Elseif first doesn't match,prev==True, and second matches then 'E'
        elif cur[1]!=token[0] and cur[2]==token[1] and prev:
            labels.append("E-" + entity_labels[cur[0]])
            #prev=False and pop label
            prev =False
            temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
            while temp[1]==cur[1]:
                cur = temp
                temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
            cur = temp
    
    return labels

            
    

In [176]:
#BOI
def BOI_labels(lab_map,offset_mapping,entity_labels):
    labels =[]
    cur = lab_map.pop()
    prev=False
    for token in offset_mapping:
        #Case:B - if first match
        if cur[1]==token[0]:
            labels.append("B-" + entity_labels[cur[0]])
            #if second doesnt match prev =True
            if cur[2]!=token[1]:
                prev = True
            else:
                #pop label mappings
                temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
                while temp[1]==cur[1]:
                    cur = temp
                    temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
                cur = temp
        #Case:I - if first doesnt match and prev==True
        elif cur[1]!=token[0] and prev:
            labels.append("I-" + entity_labels[cur[0]])
            #if second matches, prev = False and pop label mappings
            if cur[2]==token[1]:
                prev = False
                #pop labels
                temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
                while temp[1]==cur[1]:
                    cur = temp
                    temp = lab_map.pop() if len(lab_map)!=0 else ['dummy',-1,-1]
                cur = temp
        #Case:O - if first doesn't match and prev == False
        elif cur[1]!=token[0] and not prev:
            labels.append("O")
    
    return labels
        
    

In [101]:
labs = BOIES_labels(tsv_data.values.tolist(),encodings,entity_labels)

In [196]:
pd.set_option('display.max_rows', None)
i=2
df = pd.DataFrame({
    'tokens': tokenizer.convert_ids_to_tokens(encodings.input_ids[i]),
    'labels':encodings.labels[i]
})
df

,tokens,labels
0,[CLS],O
1,Power,O
2,s,O
3,##pect,O
4,##ra,O
5,of,O
6,surface,O
7,elevation,O
8,for,O
9,a,O


In [174]:
for i,j in zip(sorted(all_files),sorted(tsv_files)):
    print(i,j)

S0006322312001096-1136.txt .ipynb_checkpoints
S0006322312001096-1177.txt S0006322312001096-1136.tsv
S0006322312001096-1190.txt S0006322312001096-1177.tsv
S0006322312001096-1194.txt S0006322312001096-1190.tsv
S0006322312001096-1197.txt S0006322312001096-1194.tsv
S0006322312001096-1202.txt S0006322312001096-1197.tsv
S0006322312001096-1205.txt S0006322312001096-1202.tsv
S0006322312001096-1221.txt S0006322312001096-1205.tsv
S0006322312001096-1230.txt S0006322312001096-1221.tsv
S0006322312001096-1248.txt S0006322312001096-1230.tsv
S0006322312001096-1253.txt S0006322312001096-1248.tsv
S0006322312001096-1260.txt S0006322312001096-1253.tsv
S0006322312001096-1271.txt S0006322312001096-1260.tsv
S0006322312001096-1275.txt S0006322312001096-1271.tsv
S0006322312001096-1278.txt S0006322312001096-1275.tsv
S0006322312001096-626.txt S0006322312001096-1278.tsv
S0012821X12004384-1148.txt S0006322312001096-626.tsv
S0012821X12004384-1178.txt S0012821X12004384-1148.tsv
S0012821X12004384-1221.txt S0012821X12